<a href="https://colab.research.google.com/github/dashatenoff/recsys-vk/blob/main/recsys_hw1_toppop_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# VK RecSys HW-1 — TopPop v2 (filtered)

Improved Top Popular baseline.
- Implicit feedback (views)
- Filters out items already seen by the user
- Generates submission file


In [2]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
import os
os.listdir('/content/drive/MyDrive')
train = pd.read_parquet('/content/drive/MyDrive/VK/train.parquet')
test = pd.read_parquet('/content/drive/MyDrive/VK/test.parquet')

top_items =  train['item_id'].value_counts().head(10).index.tolist()

Mounted at /content/drive


In [7]:
user_history = (
    train.groupby('user_id')['item_id'].apply(set)
)

test_users = test['user_id'].unique()
len(test_users)

recs = []

for user_id in test_users:
  seen_items = user_history.get(user_id, set())
  for item_id in top_items:
    if item_id not in seen_items:
      recs.append({
          'user_id' : user_id,
          'recs' : item_id
      })

submission = pd.DataFrame(recs)
submission.head(10)
submission.to_csv('submission.csv', index=False)

In [8]:
submission.groupby('user_id')['recs'].size().describe()

,recs
count,8497.000000
mean,5.308815
std,1.818755
min,1.000000
25%,4.000000
50%,5.000000
75%,7.000000
max,10.000000


In [15]:
gt = test.groupby('user_id')['item_id'].apply(set)
pred = submission.groupby('user_id')['recs'].apply(list)

def average_precision_at_k(recs, relevant, k=10):
  if not relevant:
    return 0.0

  score = 0.0
  hits = 0

  for i, item in enumerate(recs[:k], start = 1):
    if item in relevant:
      hits+=1
      score+= hits/i

  return score/min(len(relevant), k)




In [16]:
some_user = pred.index[0]
ap_example = average_precision_at_k(
    pred[some_user],
    gt.get(some_user, set()),
    k=10
)
ap_example


0.03333333333333333

In [1]:
map_v1 = 0.0072870413800291845
map_v2 = 0.014914204079497713

print(f"TopPop v1 MAP@10: {map_v1:.4f}")
print(f"TopPop v2 MAP@10: {map_v2:.4f}")
print(f"Improvement: {(map_v2 / map_v1 - 1) * 100:.1f}%")


TopPop v1 MAP@10: 0.0073
TopPop v2 MAP@10: 0.0149
Improvement: 104.7%


## Model comparison

TopPop v2 filters already seen items, which leads to a significant improvement in MAP@10.
